# Loading neurotransmitter and creating connectivity subgraphs for specific neurotransmitters
### Here we build upon the previous script to take putative neurotransmitters for each connection between neurons and make subgraphs that contain one or two neurotransmitters. 

In [ ]:
from cedne import utils
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
if not os.path.isdir (utils.OUTPUT_DIR):
    os.makedirs(utils.OUTPUT_DIR)

In [ ]:
w = utils.makeWorm(chem_only=True)
nn = w.networks["Neutral"]
utils.loadNeurotransmitters(nn)

In [ ]:
conn.weight

In [ ]:
nt_count = {0:0}
lig_count = {0:0}
for e, conn in nn.connections.items():
    if hasattr(conn, 'putative_neurotrasmitter_receptors'):
        if not len(conn.putative_neurotrasmitter_receptors) in nt_count:
            nt_count[len(conn.putative_neurotrasmitter_receptors)] = 0
        nt_count[len(conn.putative_neurotrasmitter_receptors)]+=1
    else:
        nt_count[0]+=1
    
    ligc=0
    if hasattr(conn, 'ligands') and hasattr(conn, 'receptors'):
        for l in conn.ligands:
            if l in conn.receptors.values():
                ligc+=1
        if not ligc in lig_count:
            lig_count[ligc] = 0
        lig_count[ligc]+=1
    else:
        lig_count[0]+=1 

In [ ]:
num_nt, count_nt = zip(*sorted(nt_count.items(), key=lambda x:x[0]))

In [ ]:
num_lig, count_lig = zip(*sorted(lig_count.items(), key=lambda x:x[0]))

In [ ]:
f, ax = plt.subplots(figsize=(2.5,2.5), layout='constrained')
ax.scatter(num_nt, np.cumsum(count_nt)/sum(count_nt), color='gray')
utils.simpleaxis(ax)
ax.set_xlabel("Number of putative\n NT-receptor types")
ax.set_ylabel("Fraction of connections")
plt.savefig(utils.OUTPUT_DIR + "/putative-neurotransmitter-receptor-pairs-histogram.svg", transparent=True)
plt.show()

In [ ]:
lenput = []
wt = []
for e, conn in nn.connections.items():
    if hasattr(conn, 'putative_neurotrasmitter_receptors'):
        lenput.append(len(conn.putative_neurotrasmitter_receptors))
        wt.append(conn.weight)

In [ ]:
plt.scatter(lenput, wt)
plt.show()

In [ ]:
count_lig/np.sum(count_lig)

### Fold the network by neuron types (DB, VA, VC, etc.). This automatically copies the data from each of the neurons in the class to the neuron that represents the class. This aids in comaparing different sources of datasets with different formats with each other.

In [ ]:
nn_class = utils.foldByNeuronType(nn)

In [ ]:
nn_class.neurons['CAN'].type = 'interneuron' ## For the purposes of plotting

In [ ]:
conns = []
nt_rec = ('Glutamate', 'glc-2')
connFilter = 'chemical-synapse' #'gap-junction'
for c,e in nn_class.connections.items():
    if connFilter:
        if e.edge_type == connFilter:
            #print(e.putative_neurotrasmitter_receptors)
            if nt_rec in e.putative_neurotrasmitter_receptors:
                conns.append([c[0].name, c[1].name])

In [ ]:
pos = utils.plot_layered(conns, nn_class, nodeColors={}, edgeColors = 'gray', save=False, title=nt_rec, extraNodes=[], extraEdges=[], pos=[], mark_anatomical=False, colorbar=False)

### Plotting all glutamatergic connections with differnet receptor### Plotting all putative glutamatergic connections with differnet receptors.

In [ ]:
conns = {}
ligands= ('Glutamate')
connFilter = 'chemical-synapse' #'gap-junction'
for c,e in nn_class.connections.items():
    if connFilter:
        if e.edge_type == connFilter:
            #print(e.putative_neurotrasmitter_receptors)
            if ligands in e.ligands:
                for nt_rec in e.putative_neurotrasmitter_receptors:
                    if nt_rec[0] in ligands:
                        if not nt_rec in conns:
                            conns[nt_rec] = []
                        conns[nt_rec].append([c[0].name, c[1].name])

In [ ]:
for ntr, conn in conns.items():
    print(ntr)
    pos = utils.plot_layered(conn, nn_class, nodeColors={}, edgeColors = 'gray', save=False, title=ntr, extraNodes=[], extraEdges=[], pos=[], mark_anatomical=False, colorbar=False)